In [ ]:
import arcpy
import os
import time

In [ ]:
#PARAMETERS
addPts = arcpy.GetParameterAsText(0)
#locator = enter dual range locator here
outputWorkspace = arcpy.GetParameterAsText(2)
arcpy.env.overwriteOutput = True
sr = arcpy.Describe(addPts).spatialReference
split_name = arcpy.Describe(addPts).name.split('.')
fishbonesFilename = 'Fishbones_' + split_name[len(split_name)-1]
fishbones = outputWorkspace + '/' + fishbonesFilename

In [ ]:
#pre-cleanup
arcpy.Delete_management('in_memory')

In [ ]:
#copy input address points to memory (this step shouldn't be necessary, but geocoding returns zero matches without copying first)
arcpy.AddMessage('\tCopying address points to memory...')
mem_addresses = arcpy.CopyFeatures_management(addPts, 'in_memory/mem_addresses')

In [ ]:
#update x,y fields of address points
arcpy.AddMessage('\tUpdating POINT_X, POINT_Y fields...')
arcpy.AddXY_management(mem_addresses)

In [ ]:
#geocode address points to centerlines
arcpy.AddMessage('\tGeocoding address points to centerlines...')
address_fields = "'Full Address' FULLADDR VISIBLE NONE"
geocode =  arcpy.geocoding.GeocodeAddresses(mem_addresses, locator, address_fields,'in_memory/memGeocode', 'STATIC')

In [ ]:
geocode =  arcpy.geocoding.GeocodeAddresses(mem_addresses, locator, address_fields, #enter output file here, 'STATIC')


In [ ]:
#build fishbones dataset
arcpy.AddMessage('\tBuilding fishbones...')
arcpy.CreateFeatureclass_management(outputWorkspace, fishbonesFilename, 'POLYLINE', spatial_reference= sr)
arcpy.AddField_management(fishbones, 'UNIQUE_ID', 'TEXT', field_length= 100)
arcpy.AddField_management(fishbones, 'GEOCODE_STATUS', 'TEXT', field_length= 10)
cursor = arcpy.da.InsertCursor(fishbones, ['UNIQUE_ID', 'GEOCODE_STATUS', 'SHAPE@'])
for row in arcpy.da.SearchCursor(geocode, ['USER_Unique_ID', 'Status', 'USER_POINT_X', 'USER_POINT_Y', 'X', 'Y']):
    unq_id = row[0]
    status = row[1]
    startX = row[2]
    startY = row[3]
    endX = row[4]
    endY = row[5]
    
    # this line creates the geometry
    line = arcpy.Polyline(arcpy.Array([arcpy.Point(startX,startY),arcpy.Point(endX,endY)]), sr)
    
    # this line inserts the geometry into the fishbones dataset
    cursor.insertRow([unq_id, status, line])
    del line
del cursor

In [ ]:
#cleanup
arcpy.Delete_management('in_memory')